In [1]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
from time import sleep
from pprint import pformat

In [2]:
login_url = 'https://shopee.vn/buyer/login?next=https%3A%2F%2Fshopee.vn%2Froman.official%3Fpage%3D01'
shop_url = 'https://shopee.vn/roman.official?page=0'

In [3]:
options = uc.ChromeOptions()

options.user_data_dir="./temp/profile"

options.set_capability(
            "goog:loggingPrefs", {"performance": "ALL"}
        )

driver = uc.Chrome(version_main=121, options=options,enable_cdp_events=True)

def mylousyprintfunction(eventdata):

    resp_url = eventdata["params"]["response"]["url"]
    request_id = eventdata["params"]["requestId"]

    print(f"Caught {resp_url}")
    # data =  driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})
    # print(pformat(data))

    
    
# set the callback to Network.dataReceived to print (yeah not much original)
driver.add_cdp_listener("Network.responseReceived", mylousyprintfunction)

driver.get(shop_url)

In [ ]:
sleep(5)

# extract requests from logs
logs_raw = driver.get_log("performance")

logs = [json.loads(lr["message"])["message"] for lr in logs_raw]


def log_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "json" in log_["params"]["response"]["mimeType"]
    )

for log in filter(log_filter, logs):
    request_id = log["params"]["requestId"]
    resp_url = log["params"]["response"]["url"]
    print(f"Caught {resp_url}")
    try: 
       res =  driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})
       print(res)
    except:
       continue